In [143]:
import pandas as pd

df1 = pd.read_csv("../Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2 = pd.read_csv("../Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3 = pd.read_csv("../Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4 = pd.read_csv("../Monday-WorkingHours.pcap_ISCX.csv")
df5 = pd.read_csv("../Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6 = pd.read_csv("../Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", encoding='latin1')
df7 = pd.read_csv("../Tuesday-WorkingHours.pcap_ISCX.csv")
df8 = pd.read_csv("../Wednesday-workingHours.pcap_ISCX.csv")

C:\Users\Dani\AppData\Local\Temp\ipykernel_20196\2608952604.py:8: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv("../Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", encoding='latin1')


In [144]:
for df in [df1, df2, df3, df4, df5, df6, df7, df8]:
    df.columns = df.columns.str.strip()

In [145]:
for i in range(1, 9):
    df = eval(f'df{i}')
    print(f'df{i}.shape: {df.shape}')

df1.shape: (225745, 85)
df2.shape: (286467, 85)
df3.shape: (191033, 85)
df4.shape: (529918, 85)
df5.shape: (288602, 85)
df6.shape: (458968, 85)
df7.shape: (445909, 85)
df8.shape: (692703, 85)


OPERACIONES SIN CONCATENAR

In [146]:
pd.set_option('display.max_columns', 100)


In [147]:
df2['Timestamp'].unique()

array(['7/7/2017 1:00', '7/7/2017 1:01', '7/7/2017 1:02', '7/7/2017 1:03',
       '7/7/2017 1:04', '7/7/2017 1:05', '7/7/2017 1:06', '7/7/2017 1:07',
       '7/7/2017 1:08', '7/7/2017 1:09', '7/7/2017 1:10', '7/7/2017 1:11',
       '7/7/2017 1:12', '7/7/2017 1:13', '7/7/2017 1:14', '7/7/2017 1:15',
       '7/7/2017 1:16', '7/7/2017 1:17', '7/7/2017 1:18', '7/7/2017 1:19',
       '7/7/2017 1:20', '7/7/2017 1:21', '7/7/2017 1:22', '7/7/2017 1:23',
       '7/7/2017 1:24', '7/7/2017 1:25', '7/7/2017 1:26', '7/7/2017 1:27',
       '7/7/2017 1:28', '7/7/2017 1:29', '7/7/2017 1:30', '7/7/2017 1:31',
       '7/7/2017 1:32', '7/7/2017 1:33', '7/7/2017 1:34', '7/7/2017 1:35',
       '7/7/2017 1:36', '7/7/2017 1:37', '7/7/2017 1:38', '7/7/2017 1:39',
       '7/7/2017 1:40', '7/7/2017 1:41', '7/7/2017 1:42', '7/7/2017 1:43',
       '7/7/2017 1:44', '7/7/2017 1:45', '7/7/2017 1:46', '7/7/2017 1:47',
       '7/7/2017 1:48', '7/7/2017 1:49', '7/7/2017 1:50', '7/7/2017 1:51',
       '7/7/2017 1:52', '

In [ ]:
import pandas as pd

# 1) Lista de archivos (puedes ajustar paths)
files = {
    "df1_friday_ddos": "../Friday-WorkingHours-Afternoon-DDoS.pcap_ISCX.csv",
    "df2_friday_portscan": "../Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "df3_friday_morning": "../Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "df4_monday": "../Monday-WorkingHours.pcap_ISCX.csv",
    "df5_thursday_infiltration": "../Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "df6_thursday_webattacks": "../Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "df7_tuesday": "../Tuesday-WorkingHours.pcap_ISCX.csv",
    "df8_wednesday": "../Wednesday-workingHours.pcap_ISCX.csv",
}

# 2) Formatos candidatos (los más comunes en CICIDS2017)
CANDIDATE_FORMATS = [
    "%m/%d/%Y %H:%M",      # 7/7/2017 3:30
    "%m/%d/%Y %H:%M:%S",   # 7/7/2017 3:30:05
    "%d/%m/%Y %H:%M",      # 7/7/2017 3:30 (interpretación DMY)
    "%d/%m/%Y %H:%M:%S",   # 7/7/2017 3:30:05 (DMY)
    "%Y-%m-%d %H:%M:%S",   # 2017-07-07 03:30:05
    "%Y-%m-%d %H:%M",      # 2017-07-07 03:30
]

def parse_timestamp_best(series: pd.Series):
    """
    Intenta varios formatos y se queda con el que produce menos NaT.
    Devuelve (dt, best_format, nat_count).
    """
    s = series.astype(str).str.strip()

    best = None
    best_fmt = None
    best_nat = None

    for fmt in CANDIDATE_FORMATS:
        dt = pd.to_datetime(s, format=fmt, errors="coerce")
        nat = dt.isna().sum()
        if best_nat is None or nat < best_nat:
            best, best_fmt, best_nat = dt, fmt, nat
            if best_nat == 0:
                break

    # Fallback final: parser flexible (por si hay cosas raras)
    if best_nat is None:
        best = pd.to_datetime(s, errors="coerce")
        best_fmt = "infer"
        best_nat = best.isna().sum()

    return best, best_fmt, best_nat


report_rows = []
dfs = {}

for name, path in files.items():
    # OJO con encoding en el que ya sabes que falla
    if "webattacks" in name:
        df = pd.read_csv(path, encoding="latin1")
    else:
        df = pd.read_csv(path)

    # Limpia nombres de columnas por si hay espacios tipo ' Timestamp'
    df.columns = df.columns.str.strip()

    if "Timestamp" not in df.columns:
        raise ValueError(f"{name}: no encuentro columna 'Timestamp'. Columnas: {df.columns.tolist()[:10]} ...")

    df["dt"], best_fmt, nat_count = parse_timestamp_best(df["Timestamp"])

    report_rows.append({
        "df": name,
        "rows": len(df),
        "best_format": best_fmt,
        "NaT_count": int(nat_count),
        "NaT_%": float(nat_count / len(df) * 100),
        "min_dt": df["dt"].min(),
        "max_dt": df["dt"].max(),
        "unique_days": int(df["dt"].dt.date.nunique(dropna=True)),
    })

    dfs[name] = df

report = pd.DataFrame(report_rows).sort_values("df")
print(report)


C:\Users\Dani\AppData\Local\Temp\ipykernel_20196\732503039.py:59: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding="latin1")


                          df    rows        best_format  NaT_count      NaT_%  \
0            df1_friday_ddos  225745     %m/%d/%Y %H:%M          0   0.000000   
1        df2_friday_portscan  286467     %m/%d/%Y %H:%M          0   0.000000   
2         df3_friday_morning  191033     %m/%d/%Y %H:%M          0   0.000000   
3                 df4_monday  529918  %m/%d/%Y %H:%M:%S          0   0.000000   
4  df5_thursday_infiltration  288602     %m/%d/%Y %H:%M          0   0.000000   
5    df6_thursday_webattacks  458968     %m/%d/%Y %H:%M     288602  62.880637   
6                df7_tuesday  445909     %m/%d/%Y %H:%M          0   0.000000   
7              df8_wednesday  692703     %m/%d/%Y %H:%M          0   0.000000   

               min_dt              max_dt  unique_days  
0 2017-07-07 03:30:00 2017-07-07 05:02:00            1  
1 2017-07-07 01:00:00 2017-07-07 03:29:00            1  
2 2017-07-07 08:59:00 2017-07-07 12:59:00            1  
3 2017-03-07 01:00:01 2017-03-07 12:59:58 

In [148]:
s = df6["Timestamp"].astype(str).str.strip()

dt1 = pd.to_datetime(s, format="%m/%d/%Y %H:%M", errors="coerce")
dt2 = pd.to_datetime(s, format="%m/%d/%Y %H:%M:%S", errors="coerce")

df6["dt"] = dt1.fillna(dt2)

print("NaT:", df6["dt"].isna().sum())
print(df6["dt"].min(), df6["dt"].max())

NaT: 288602
2017-06-07 08:59:00 2017-06-07 12:59:00


BORRAR NAN DF 6

In [149]:
df6 = df6.dropna(subset=["Timestamp"]).copy()

In [150]:
df6["Timestamp"].isna().sum()
df6.shape

(170366, 86)

In [151]:
df6['Timestamp'].unique()

array(['6/7/2017 8:59', '6/7/2017 9:00', '6/7/2017 9:01', '6/7/2017 9:02',
       '6/7/2017 9:03', '6/7/2017 9:04', '6/7/2017 9:05', '6/7/2017 9:06',
       '6/7/2017 9:07', '6/7/2017 9:08', '6/7/2017 9:09', '6/7/2017 9:10',
       '6/7/2017 9:11', '6/7/2017 9:12', '6/7/2017 9:13', '6/7/2017 9:14',
       '6/7/2017 9:15', '6/7/2017 9:16', '6/7/2017 9:17', '6/7/2017 9:18',
       '6/7/2017 9:19', '6/7/2017 9:20', '6/7/2017 9:21', '6/7/2017 9:22',
       '6/7/2017 9:23', '6/7/2017 9:24', '6/7/2017 9:25', '6/7/2017 9:26',
       '6/7/2017 9:27', '6/7/2017 9:28', '6/7/2017 9:29', '6/7/2017 9:30',
       '6/7/2017 9:31', '6/7/2017 9:32', '6/7/2017 9:33', '6/7/2017 9:34',
       '6/7/2017 9:35', '6/7/2017 9:36', '6/7/2017 9:37', '6/7/2017 9:38',
       '6/7/2017 9:39', '6/7/2017 9:40', '6/7/2017 9:41', '6/7/2017 9:42',
       '6/7/2017 9:43', '6/7/2017 9:44', '6/7/2017 9:45', '6/7/2017 9:46',
       '6/7/2017 9:47', '6/7/2017 9:48', '6/7/2017 9:49', '6/7/2017 9:50',
       '6/7/2017 9:51', '

COMIENZA LIMPIEZA TIMESTAMP DF A DF

In [ ]:
def convert_timestamp_standard(df):
    df["dt"] = pd.to_datetime(
        df["Timestamp"],
        format="%d/%m/%Y %H:%M",
        errors="coerce"
    )
    return df


Operar con df4 que parece que da problemas con la conversión

In [154]:
df4[["Timestamp"]].head()   # si esto da KeyError, NO existe "Timestamp"



,Timestamp
0,03/07/2017 08:55:58
1,03/07/2017 08:55:58
2,03/07/2017 08:55:58
3,03/07/2017 08:55:58
4,03/07/2017 08:56:22


In [155]:
df4["dt"] = pd.to_datetime(
    df4["Timestamp"],
    format="%m/%d/%Y %H:%M:%S",
    errors="coerce"
)

print("df4 NaT:", df4["dt"].isna().sum())
print(df4["dt"].min(), df4["dt"].max())

df4 NaT: 0
2017-03-07 01:00:01 2017-03-07 12:59:58


In [156]:
df1 = convert_timestamp_standard(df1)
df2 = convert_timestamp_standard(df2)
df3 = convert_timestamp_standard(df3)
#df4 = convert_timestamp_standard(df4)
df5 = convert_timestamp_standard(df5)
df6 = convert_timestamp_standard(df6)
df7 = convert_timestamp_standard(df7)
df8 = convert_timestamp_standard(df8)


In [157]:
for name, df in zip(
    ["df1","df2","df3","df4","df5","df7","df8"],
    [df1,df2,df3,df4,df5,df7,df8]
):
    print(name, "NaT:", df["dt"].isna().sum())


df1 NaT: 0
df2 NaT: 0
df3 NaT: 0
df4 NaT: 0
df5 NaT: 0
df7 NaT: 0
df8 NaT: 0


In [158]:
pd.isna(df1['dt']).sum()


0

In [159]:
pd.isna(df2['dt']).sum()


0

In [160]:
pd.isna(df3['dt']).sum()


0

In [161]:
pd.isna(df4['dt']).sum()


0

In [162]:
pd.isna(df5['dt']).sum()


0

In [163]:
pd.isna(df6['dt']).sum()


0

In [164]:
pd.isna(df7['dt']).sum()


0

In [165]:
pd.isna(df8['dt']).sum()

0

In [166]:
data_list = [df1, df2, df3, df4, df5, df6, df7, df8]
df = pd.concat(data_list, ignore_index=True)

rows, cols = df.shape
print('New dimension:')
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')

New dimension:
Number of rows: 2830743
Number of columns: 86
Total cells: 243443898


In [167]:
df['Timestamp'].isna().sum()

0

In [168]:
dfTimestamp = df[['Timestamp']]

type(dfTimestamp)

pandas.core.frame.DataFrame

In [169]:
dfTimestamp.shape

(2830743, 1)

In [170]:
dups = dfTimestamp[dfTimestamp.duplicated()]
len(dups)

2802778